In [2]:
import pandas as pd
import numpy as np
import csv
import os
import re
import nltk
import itertools
import contractions
import pickle

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('words')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.metrics.distance import edit_distance
from nltk.corpus import words

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from spellchecker import SpellChecker

import matplotlib.pyplot as plt
import seaborn as sns

#os.chdir('Documents/forum-classifier-python')
print(os.getcwd())

/Users/mayaechtinaw/Documents/forum-classifier-python


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mayaechtinaw/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mayaechtinaw/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/mayaechtinaw/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/mayaechtinaw/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/mayaechtinaw/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [3]:
with open('trained_classifier.pkl', 'rb') as file:
    vectorizer = pickle.load(file)
    classifier = pickle.load(file)

In [4]:
input_csv = input("Enter an input csv file: ")
data = pd.read_csv(input_csv)

Enter an input csv file: larger_data.csv


In [5]:
contractions = {
    "ain't": "am not / are not",
    "aren't": "are not / am not",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he had / he would",
    "he'd've": "he would have",
    "he'll": "he shall / he will",
    "he'll've": "he shall have / he will have",
    "he's": "he has / he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how has / how is",
    "I'd": "I had / I would",
    "I'd've": "I would have",
    "I'll": "I shall / I will",
    "I'll've": "I shall have / I will have",
    "I'm": "I am",
    "I've": "I have",
    "isn't": "is not",
    "it'd": "it had / it would",
    "it'd've": "it would have",
    "it'll": "it shall / it will",
    "it'll've": "it shall have / it will have",
    "it's": "it has / it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she had / she would",
    "she'd've": "she would have",
    "she'll": "she shall / she will",
    "she'll've": "she shall have / she will have",
    "she's": "she has / she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as / so is",
    "that'd": "that would / that had",
    "that'd've": "that would have",
    "that's": "that has / that is",
    "there'd": "there had / there would",
    "there'd've": "there would have",
    "there's": "there has / there is",
    "they'd": "they had / they would",
    "they'd've": "they would have",
    "they'll": "they shall / they will",
    "they'll've": "they shall have / they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we had / we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what shall / what will",
    "what'll've": "what shall have / what will have",
    "what're": "what are",
    "what's": "what has / what is",
    "what've": "what have",
    "when's": "when has / when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where has / where is",
    "where've": "where have",
    "who'll": "who shall / who will",
    "who'll've": "who shall have / who will have",
    "who's": "who has / who is",
    "who've": "who have",
    "why's": "why has / why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'alls": "you alls",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you had / you would",
    "you'd've": "you would have",
    "you'll": "you shall / you will",
    "you'll've": "you shall have / you will have",
    "you're": "you are",
    "you've": "you have"
}

contraction_pattern = re.compile(r'\b(' + '|'.join(contractions.keys()) + r')\b')

def expand_contractions(data, contraction_mapping=contractions):
    def expand_match(contraction):
        match = contraction.group(0)
        expanded_contraction = contraction_mapping.get(match)
        if not expanded_contraction:
            expanded_contraction = contraction_mapping.get(match.lower())
        return expanded_contraction

    expanded_data = contraction_pattern.sub(expand_match, data)
    return expanded_data

def remove_word(data, word):
    return data.replace(word, '')

def remove_URL(data):
    url_pattern = re.compile(r'https://\S+|www\.\S+')
    return url_pattern.sub(r'', data)

def remove_HTML(data):
    html_tags_pattern = r'<.*?>'
    return re.sub(html_tags_pattern, '', data)

def preprocess_text(data):
    stop_words = set(stopwords.words('english'))  # Define stop words
    stemmer = PorterStemmer()  # Define stemmer
    lemmatizer = WordNetLemmatizer()  # Define lemmatizer
    spell = SpellChecker()  # Define spell checker
    data['content'] = data['content'].apply(expand_contractions)  # Apply contraction expansion
    data['content'] = data['content'].apply(lambda x: remove_URL(x))  # Remove URLs
    data['content'] = data['content'].apply(lambda x: remove_HTML(x))  # Remove HTML tags
    data['content'] = data['content'].str.lower()  # Lowercase all words
    data['content'] = data['content'].apply(lambda x: re.sub(r'[^a-zA-Z0-9 \s]', ' ', x))  # Remove special characters
    data['content'] = data['content'].apply(lambda x: remove_word(x, 'quote'))
    data['content'] = data['content'].str.split().map(lambda sl: " ".join(s for s in sl if len(s) > 2))  # Remove words with length < 2

    
    #data['tokenized'] = data.apply(lambda row: nltk.word_tokenize(row['content']), axis=1)  # Tokenizing
    #data['tokenized'] = data['tokenized'].apply(lambda x: [item for item in x if item not in stop_words])  # Remove stop words
    #data['tokenized'] = data['tokenized'].apply(lambda x: [stemmer.stem(y) for y in x])  # Stemming
    #data['tokenized'] = data['tokenized'].apply(lambda x: [lemmatizer.lemmatize(y) for y in x])  # Lemmatizing
    #data['clean_string'] = data['tokenized'].str.join(' ')  # De-tokenizes string
    return data

data = preprocess_text(data)

print(data.head)

<bound method NDFrame.head of                                               content
0         entry door step seal not installed properly
1           reglued the end seals the full wall slide
2   got temp plates friday passed emissions friday...
3   had the airbags replaced after 20yrs figured w...
4   hmm two new chassis batteries new service aqua...
..                                                ...
70                         working sewer hose storage
71                                            content
72  took all winter but had replace throttle link ...
73  realigned wiper arm parked too low the windshi...
74  far this spring removed old tube last year and...

[75 rows x 1 columns]>


In [6]:
X_new_tfidf = vectorizer.transform(data['content'])

In [7]:
predictions = classifier.predict(X_new_tfidf)

In [8]:
data['prediction'] = predictions

In [9]:
product_mapping = {
    0: 'Chassis',
    1: 'Door Locks and Parts',
    2: 'Electrical',
    3: 'Exterior',
    4: 'Hardware',
    5: 'Heating and Cooling',
    6: 'Lights',
    7: 'Plumbing and Water',
    8: 'Wiper Parts',
    9: 'Adhesives, Sealants, and Cleaners',
    10: 'Winterizing',
    11: 'Interior'
}

data['prediction'] = data['prediction'].apply(lambda x: product_mapping[x])

In [10]:
data.to_csv('forum_output.csv', index=False)

In [13]:
print(data.head)

<bound method NDFrame.head of                                               content            prediction
0         entry door step seal not installed properly  Door Locks and Parts
1           reglued the end seals the full wall slide              Exterior
2   got temp plates friday passed emissions friday...                Lights
3   had the airbags replaced after 20yrs figured w...              Exterior
4   hmm two new chassis batteries new service aqua...    Plumbing and Water
..                                                ...                   ...
70                         working sewer hose storage    Plumbing and Water
71                                            content              Exterior
72  took all winter but had replace throttle link ...              Exterior
73  realigned wiper arm parked too low the windshi...              Exterior
74  far this spring removed old tube last year and...    Plumbing and Water

[75 rows x 2 columns]>
